# Casos de uso

In [51]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def classify(examples, request):
    prompt = examples + "\n" + request + ":"
    restart_sequence = "\n"
    response = openai.Completion.create(
      engine="davinci",
      prompt=prompt,
      temperature=0,
      max_tokens=30,
      logprobs=5,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
    )
    return response

# Identificación del dominio

In [96]:
import math
import numpy as np

examples = "Hola|SALUDAR\n\start|SALUDAR\nHola, soy Diego|SALUDAR\nMe tengo que tomar el ibuprofeno a las 3|MEDICINA\n"
output = classify(examples, "Hola|")
print(output["choices"][0]["text"])
probs = output["choices"][0]["logprobs"]["token_logprobs"]
p = []
for prob in probs:
    p.append(round(math.exp(prob)*100, 2))
print(probs)
print(p)
print(round(np.mean(p), 2))

SALUDAR
[-1.6015434, -0.004846714, -0.009111802, -0.009979544, -0.21895282, -1.8166513, -0.38993603, -0.58230245]
[20.16, 99.52, 99.09, 99.01, 80.34, 16.26, 67.71, 55.86]
67.24


# Saludo

In [41]:
examples = "Human: START\nRobot: Hola Human\nRobot: ¿qué tal estás?\nHuman: ¡Hola!\nRobot: ¿Qué tal estás, Human?"
output = classify(examples, "Diego: ¡Hola!\nRobot:")
print(output)

 ¿Qué tal estás, Diego?


## Detección del estado

In [2]:
examples = "Lista de textos:\ntengo frío, hace frío:DETECTAR_FRIO|medio|¿enciendo la calefacción?.\nlate muy fuerte, está muy alto el pulso:DETECTAR_PULSO_ALTO|alto|Relájate."
#output = classify(examples, "Estoy muy nervioso")
#output = classify(examples, "No me siento bien")
#output = classify(examples, "Ese payaso me da miedo.")
output = classify(examples, "Estoy de bajón.")
print(output)

DETECTAR_BAJON|medio|¿Quieres que te prepare un té?.


## Recordatorio medicamentos

In [4]:
examples = "Tengo que tomar un ibuprofeno los lunes|TOMAR#IBUPROFENO|LUNES.\nA las 12:00 me toca el paracetamol|TOMAR#PARACETAMOL|12:00."
output = classify(examples, "A las 8 calcio.")
print(output)
output = classify(examples, "Me ha dicho el médico que me tome la pastilla de la alergia cuando me levante.")
print(output)

|TOMAR#CALCIO|8:00.
|TOMAR#ALERGIA|LEVANTARSE.


In [10]:
examples = "TOMAR#IBUPROFENO|lunes.TOMAR#PARACETAMOL|12:00.\nTOMAR#CALCIO:10:00.\nTOMAR#ALERGIAPASTILLA:al levantarme\n¿cuánto me toca el IBUPROFENO?:los lunes.\n¿por qué me pasa esto?:porque te pasa razón."
def test(pregunta):
    print(pregunta)
    output = classify(examples, pregunta)
    print(output)

test("¿cuándo me tengo que tomar el calcio?")
test("¿a qué hora era lo del paracetamol?")
test("¿a qué hora me tengo que tomar el ibuprofeno?")
test("¿cuándo me tengo que tomar el ibuprofeno?")
test("¿por qué me tengo que tomar el ibuprofeno?")

¿cuándo me tengo que tomar el calcio?
los lunes.
¿a qué hora era lo del paracetamol?
a las 12:00.
¿a qué hora me tengo que tomar el ibuprofeno?
los lunes.
¿cuándo me tengo que tomar el ibuprofeno?
los lunes.
¿por qué me tengo que tomar el ibuprofeno?
porque te pasa razón.


## Cuaderno de bitácora

In [6]:
examples = "El cumple de María es el 03/04/1993:Cumpleaños|María|03/04/1993.\nEl aniversario boda de Pepa y Pepe es el 10 de octubre de 2014:Aniversario boda|Pepa y Pepe|10/10/2014.\nPepe cumple años el 12 de diciembre de 2001:Cumpleaños|Pepe|12/12/2010."
output = classify(examples, "¿cuándo es el aniversario de Pepa y Pepe?")
print(output)

Aniversario boda|Pepa y Pepe|10/10/2014.


In [11]:
examples = "Persona: Tengo dentista el día 15 a las 8.\nRobot: Lo anoto#anotar|dentista|15/CurrentMonth/CurrentYear 08:00.\nHumano: A qué hora tengo dentista??.\nRobot: A las 8:00 el día 15 de enero de 2022.\nCron: Recuerda a Persona que tiene dentista mañana.\nRobot: Persona, mañana tienes dentista a las 8:00.\nSensor temperatura: hace frío.\nRobot: Hace frío, quieres que encienda la calefacción??#confirmacion|encender|calefacción.\nHumano: sí.\nRobot: ok#confirmado.\nSensor de conexión: inactividad|baja.\nRobot: Hola, qué tal?.\nSensor de conexión: inactividad|alta.\nRobot: Voy a avisar a alguien de tu familia de que no contestas en 10 minutos#avisofamilia|10.\nPersona: estoy bien."
print("Tengo que ir al otorrino")
output = classify(examples, "Persona: Tengo que ir al otorrino el día 30 a las 10.\nRobot: .")
print(output)
print("Cron recuerda que tienes dentista")
output = classify(examples, "Cron: Dile a Pepe que tiene que ir al otorrino el lunes.\nRobot: .")
print(output)

Tengo que ir al otorrino
|otorrino|30/CurrentMonth/CurrentYear 10:00.
Cron recuerda que tienes dentista
|pepe|otorrino|lunes|dentista|15/CurrentMonth/CurrentYear 08:00|15/CurrentMonth
